In [7]:
import tensorflow as tf 
from helper_functions import walk_through_dir
import os

In [8]:
## Creating Helper functions
from helper_functions import create_tensorboard_callback, plot_loss_curves, accuracy_score,compare_historys,confusion_matrix, walk_through_dir

In [9]:
walk_through_dir("101_food_classes_10_percent")

here
There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent\test'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\apple_pie'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\baby_back_ribs'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\baklava'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\beef_carpaccio'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\beef_tartare'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\beet_salad'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\beignets'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\bibimbap'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\bread_pudding'.
There are 0 directories and 250 images i

In [10]:
train_dir = "101_food_classes_10_percent/train"
test_dir = "101_food_classes_10_percent/test"
IMAGE_SIZE = (224,224)

In [11]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode='categorical',
                                                                 image_size=IMAGE_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode='categorical',
                                                                image_size=IMAGE_SIZE,
                                                                shuffle=False)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


## Train a big model with transfer learning on 10% of all food 101 data 

Other Steps.
1. Create a Model Checkpoint callback
2. Create a data augumentation layer to build data augumentation right into the model 
3. build a headless ( No Top Layers ) Functional EfficientNetB0 backboned model. ( we'll create our own output layer)
4. Compile our model
5. Feature Extraction for 5 full passess ( 5 Epcohs on the train dataset and validate on 15% of the test data to save epcochs time)

### 1. Create a Model CheckPoint Callback

In [12]:
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor='val_accuracy',
                                                         save_best_only=True)

### 2. Create a data augumentation layer to build data augumentation right into the model 

In [13]:
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [14]:
data_augmentation = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    # preprocessing.Rescaling(1/255.) All the EfficientNet models have preprocessing build in so we don't need to add this here
],name="data_augmentation")

In [15]:
# Setup the base model and freeze its layers
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Setup model architecture with trainable top layers
inputs = layers.Input(shape=(224,224,3), name="input_layer")
x = data_augmentation(inputs) # Augument Images( Only Happes during Training )
x = base_model(x, training=False) # put the base model in infrence mode so weights which need to stay frozen, stay frozen
x = layers.GlobalAveragePooling2D(name='global_avg_pool_layer')(x)
outputs = layers.Dense(len(train_data.class_names), activation='softmax',name='output_layer')(x)

model = tf.keras.Model(inputs,outputs)

In [16]:
# Get the summary of the model we created
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_avg_pool_layer (Glo  (None, 1280)              0         
 balAveragePooling2D)                                            
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                             

In [17]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
                      loss=tf.keras.losses.categorical_crossentropy,
                      metrics=['accuracy'])
    
history = model.fit(train_data,
                    epochs=1,
                     validation_data=test_data,
                     validation_steps=int(0.15*(len(test_data))),
                     callbacks=[checkpoint_callback])

  4/237 [..............................] - ETA: 11:21 - loss: 4.6718 - accuracy: 0.0156